## Random Forest Regression
https://www.geeksforgeeks.org/random-forest-regression-in-python/

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings 
import numpy as np
 
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
 
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('Resources/clean_realestate_data.csv', encoding='ascii')
df.head()

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size
0,for_sale,525000.0,3.0,3.0,0.45,Agawam,Massachusetts,1001.0,2314.0
1,for_sale,289900.0,3.0,2.0,0.36,Agawam,Massachusetts,1001.0,1276.0
2,for_sale,384900.0,3.0,2.0,0.46,Agawam,Massachusetts,1001.0,1476.0
3,for_sale,199999.0,3.0,2.0,1.76,Agawam,Massachusetts,1001.0,1968.0
4,for_sale,419000.0,4.0,2.0,2.00,Pelham,Massachusetts,1002.0,1607.0


In [11]:
# reorder the columns
df = df[['status', 'bed', 'bath','acre_lot','zip_code','house_size', 'price']]
df.head()

,status,bed,bath,acre_lot,zip_code,house_size,price
0,for_sale,3.0,3.0,0.45,1001.0,2314.0,525000.0
1,for_sale,3.0,2.0,0.36,1001.0,1276.0,289900.0
2,for_sale,3.0,2.0,0.46,1001.0,1476.0,384900.0
3,for_sale,3.0,2.0,1.76,1001.0,1968.0,199999.0
4,for_sale,4.0,2.0,2.00,1002.0,1607.0,419000.0


In [12]:
# Set the features and the target
X = df.iloc[:,1:6].values  
y = df.iloc[:,6].values  

In [14]:
label_encoder = LabelEncoder()
x_categorical = df.select_dtypes(include=['object']).apply(label_encoder.fit_transform)
x_numerical = df.select_dtypes(exclude=['object']).values
x = pd.concat([pd.DataFrame(x_numerical), x_categorical], axis=1).values
 
# Fitting Random Forest Regression to the dataset
regressor = RandomForestRegressor(n_estimators=10, random_state=0, oob_score=True)
 
# Fit the regressor with x and y data
regressor.fit(x, y)


RandomForestRegressor(n_estimators=10, oob_score=True, random_state=0)

In [15]:

# Evaluating the model
from sklearn.metrics import mean_squared_error, r2_score
 
# Access the OOB Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')
 
# Making predictions on the same data or new data
predictions = regressor.predict(x)
 
# Evaluating the model
mse = mean_squared_error(y, predictions)
print(f'Mean Squared Error: {mse}')
 
r2 = r2_score(y, predictions)
print(f'R-squared: {r2}')


Out-of-Bag Score: 0.8838937458686524
Mean Squared Error: 13842884573.701181
R-squared: 0.9901397742203788
